In [ ]:
import numpy as np
import pandas as pd

def benchmark(x1, x2):
    return (    (1.3356 * (1.5 * (1 - x1))) 
                + (np.exp((2 * x1) - 1) * np.sin((3 * np.pi) * ((x1 - 0.6) ** 2)))
                + (np.exp(3 * (x2 - 0.5)) * np.sin((4 * np.pi) * ((x2 - 0.9) ** 2)))
            )

def getData(grid):
    x1 = np.linspace(0, 1, grid)
    x2 = np.linspace(0, 1, grid)
    x1, x2 = np.meshgrid(x1, x2)
    f_x1_x2 = benchmark(x1, x2)
    data = {'x1': x1.flatten(), 'x2': x2.flatten(), 'f(x1,x2)': f_x1_x2.flatten()}
    df = pd.DataFrame(data)
    return df


df_1000 = getData(32)
df_test = pd.read_excel("../Kriging-data.xlsx", sheet_name="Test")
df_training = pd.read_excel("../Kriging-data.xlsx", sheet_name="Training")
df_tudo = pd.read_excel("../Kriging-data.xlsx", sheet_name="Tudo")


df_virtual = pd.read_excel("../VirtualSamples.xlsx", sheet_name="PyKrige")

In [ ]:
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

scaler = StandardScaler()
out_scaler = StandardScaler()

def show_norm(df, label="data", plot=False):
    df_norm = pd.DataFrame(scaler.transform(df), columns=df.columns)
    df_denorm = pd.DataFrame(scaler.inverse_transform(df_norm), columns=df_norm.columns)

    if (plot):
        df.plot(title=f"{label}: Original data")
        df_norm.plot(title=f"{label}: Normalized data")
        df_denorm.plot(title=f"{label}: Denormalized data")
    return (df_norm)


def test_out_scaler(df):
    out = df["f(x1,x2)"].values.reshape(-1, 1)  
    plt.plot(out, label='Original')
    out_scaler.fit(out)
    norm = out_scaler.transform(out)
    plt.plot(norm, label='Normalizado')
    plt.plot(out_scaler.inverse_transform(norm), label='desnormalizado')
    plt.legend()
    plt.show()

scaler.fit(df_training)
test_out_scaler(df_training)

df_training_norm = show_norm(df_training, "Training", True)
df_1000_norm = show_norm(df_1000)
df_virtual_norm = show_norm(pd.concat([df_training, df_virtual]))



In [ ]:
import plotly.graph_objs as go
import plotly.offline as py
import plotly.figure_factory as ff
from scipy.spatial import Delaunay

def plot_model_comparison(df_original, df_model, model_name):
    # Dados originais
    x_orig, y_orig, z_orig = df_original['x1'], df_original['x2'], df_original['f(x1,x2)']
    x_sup, y_sup, z_sup = df_1000['x1'], df_1000['x2'], df_1000['f(x1,x2)']

    # Dados do modelo específico
    x_model, y_model, z_model = df_model['x1'], df_model['x2'], df_model['f(x1,x2)']

    # Criação da superfície
    points = np.column_stack((x_sup, y_sup))
    tri = Delaunay(points)
    fig_surface = ff.create_trisurf(x=x_sup, y=y_sup, z=z_sup,
                                    simplices=tri.simplices, 
                                    title=f'Surface Fit - Original vs {model_name} Model')

    # Pontos das amostras originais
    original_data = go.Scatter3d(x=x_orig, y=y_orig, z=z_orig, mode='markers',
                                 marker=dict(size=5, color='red', opacity=0.8),
                                 name='Original Samples')

    # Pontos do modelo específico
    model_data = go.Scatter3d(x=x_model, y=y_model, z=z_model, mode='markers',
                              marker=dict(size=5, color='blue', opacity=0.8),
                              name=f'{model_name} Model')

    # Criando a figura
    fig = go.Figure(data=[fig_surface.data[0], original_data, model_data])
    fig.update_layout(
        scene=dict(
            xaxis_title='x1',
            yaxis_title='x2',
            zaxis_title='f(x1, x2)',
            aspectmode='cube'  
        )
    )

    # Plotando o gráfico
    py.iplot(fig)

In [ ]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

In [ ]:
def split_df(df):
    _input = np.vstack([df['x1'], df['x2']]).T
    _output = np.array(df['f(x1,x2)'])
    return (_input, _output)

In [ ]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def compute_k(self):
        total_parameters = 0
        for layer in self.model.layers:
            weights = layer.get_weights()
            if len(weights) > 0:  
                for w in weights:
                    total_parameters += np.prod(w.shape)
        return total_parameters
        
    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer

        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

In [ ]:
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error, root_mean_squared_error, mean_absolute_percentage_error 

class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
        self.k = 0

    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)
      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output, sup_input, sup_output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.sup_dataset, self.sup_input, self.sup_output = self.create_dataset(sup_input, sup_output)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._data = (input, output)
      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
      self._sup = (sup_input, sup_output)

    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)
    
    def get_new_metrics(self, orig, pred, r2, mse):
      n = len(orig) # N: quantidade de saidas
      k = self.k
      waste = (orig.flatten() - pred.flatten())

      mape = mean_absolute_percentage_error(orig, pred)  
      r2_adj = 1 - (((n - 1)/(n - k - 1)) * (1 - r2))
      rsd = np.sqrt(np.sum(waste ** 2) / (n - 2))
      rmse = root_mean_squared_error(orig, pred)          
      aic = (-2 * np.log(mse)) + (2 * k)
      bic = (-2 * np.log(mse)) + (k * np.log(n))
      return (mape, r2_adj, rsd, rmse, aic, bic)
      

    def get_metrics(self):
          # Calculando a saida com os dados normalizados
          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.lm_model.predict(self.test_input).flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()
          sup_pred = self.lm_model.predict(self.sup_input).flatten()

          # Calculando as metricas com a saida desnormalizada
          pred_denorm = out_scaler.inverse_transform(pred.reshape(-1, 1))
          test_pred_denorm = out_scaler.inverse_transform(test_pred.reshape(-1, 1))
          val_pred_denorm = out_scaler.inverse_transform(val_pred.reshape(-1, 1))
          vt_pred_denorm = out_scaler.inverse_transform(vt_pred.reshape(-1, 1))
          sup_pred_denorm = out_scaler.inverse_transform(sup_pred.reshape(-1, 1))

          out_denorm = out_scaler.inverse_transform(self._data[1].reshape(-1, 1))
          test_denorm = out_scaler.inverse_transform(self._test[1].reshape(-1, 1))
          val_denorm = out_scaler.inverse_transform(self._val[1].reshape(-1, 1))
          vt_denorm = out_scaler.inverse_transform(self._vt[1].reshape(-1, 1))    
          sup_denorm = out_scaler.inverse_transform(self._sup[1].reshape(-1, 1))

          r2 = r2_score(out_denorm, pred_denorm)
          r2_test = r2_score(test_denorm, test_pred_denorm)
          r2_val = r2_score(val_denorm, val_pred_denorm)
          r2_vt = r2_score(vt_denorm,  vt_pred_denorm)
          r2_sup = r2_score(sup_denorm,  sup_pred_denorm)

          mse = mean_squared_error(out_denorm, pred_denorm)
          mse_test = mean_squared_error(test_denorm, test_pred_denorm)
          mse_val = mean_squared_error(val_denorm, val_pred_denorm)
          mse_vt = mean_squared_error(vt_denorm,  vt_pred_denorm)
          mse_sup = mean_squared_error(sup_denorm,  sup_pred_denorm)
          
          mape, r2_adj, rsd, rmse, aic, bic = self.get_new_metrics(out_denorm, pred_denorm, r2, mse)
          metrics = {
                          'r2': r2,
                          'r2_sup': r2_sup,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_sup': mse_sup,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt,
                          'mape': mape,
                          'rmse': rmse,
                          'r2_adj': r2_adj,
                          'rsd': rsd,
                          'aic': aic,
                          'bic': bic
                          }

          return metrics

In [ ]:
import pickle
from itertools import product

class Tester:
  def __init__(self, _df, _df_1000,  run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = split_df(_df)
    self.input_1000, self.output_1000 = split_df(_df_1000)
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=2,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)
    trainer.k = shuffler.compute_k()

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path)

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._data, trainer._train, trainer._vt, trainer._val, trainer._test), f)
      
  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] <= boundarie): # should be >= to acsending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    return(save)

  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output, self.input_1000, self.output_1000)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults('mse_sup', header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

# Treinando apenas com dados originais


In [9]:
tester = Tester(_df=df_virtual_norm,
                _df_1000=df_1000_norm,
                run_times=25, dataset_run_times=10)
tester.Loop(sort_by='mse_sup',
            boundarie = 0.5,
            hidden_sizes = [[40, 39], [20, 4], [15, 14], [26,18], [33, 21], [21, 12], [30, 21], [15, 2], [24, 7], [30, 16]],
            regularizers=[0.2, 0.0, 0.2, 0.5],
            learning_rate=[0.01, 0.1])

Testando combinacao1: Hidden Size=[40, 39], regularizer=0.2, learning_rate=0.01
+++++++++++ [1_0] | 1 ++++++++++++++++++
Stopped at epoch:  12
32/32 [==============================] - 0s 3ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Stopped at epoch:  6
32/32 [==============================] - 0s 5ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Stopped at epoch:  6
32/32 [==============================] - 0s 4ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Stopped at epoch:  6
32/32 [==============================] - 0s 4ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Stopped at epoch:  6
32/32 [==============================] - 0s 6ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Stopped at epoch:  6
32/32 [==============================] - 0s 6ms/step
+++++++++++ [1_0] | 7 ++++++++++++++++++
Stopped at epoch:  6
32/32 [==============================] - 0s 4ms/step
+++++++++++ [1_0] | 8 ++++++++++++++++++
Stopped at epoch:  6
32/32 [==============================] - 0s 4ms/step

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic
model_1_1_18,0.668303,0.418751,0.603170,0.655157,0.636845,0.128226,0.345054,0.130814,0.174890,0.152852,0.152300,0.358087,1.025375,0.360963,3522.107920,8511.127795
model_1_1_17,0.661572,0.413879,0.595637,0.648414,0.629833,0.130828,0.347947,0.133297,0.178310,0.155804,0.153842,0.361702,1.025890,0.364607,3522.067740,8511.087614
model_1_1_16,0.654821,0.408685,0.589090,0.641506,0.623107,0.133438,0.351030,0.135455,0.181814,0.158634,0.155416,0.365292,1.026406,0.368226,3522.028238,8511.048112
model_1_1_15,0.647835,0.403595,0.581003,0.634381,0.615648,0.136139,0.354052,0.138121,0.185427,0.161774,0.156980,0.368970,1.026940,0.371933,3521.988164,8511.008038
model_1_1_14,0.617933,0.386473,0.506443,0.613257,0.573723,0.147698,0.364216,0.162699,0.196140,0.179420,0.163966,0.384315,1.029228,0.387402,3521.825175,8510.845049
model_1_1_13,0.612245,0.382675,0.499755,0.608029,0.567955,0.149897,0.366471,0.164904,0.198792,0.181848,0.165327,0.387165,1.029663,0.390275,3521.795620,8510.815494
model_1_1_12,0.604439,0.376927,0.488835,0.600591,0.559197,0.152914,0.369883,0.168504,0.202564,0.185534,0.166893,0.391043,1.030260,0.394184,3521.755754,8510.775628
model_1_1_11,0.596460,0.371093,0.478323,0.592799,0.550387,0.155999,0.373346,0.171969,0.206516,0.189242,0.168546,0.394967,1.030871,0.398139,3521.715812,8510.735687
model_1_1_10,0.588337,0.364976,0.467674,0.584929,0.541475,0.159139,0.376978,0.175479,0.210507,0.192993,0.170217,0.398922,1.031492,0.402127,3521.675954,8510.695829
model_1_1_9,0.579912,0.358270,0.455976,0.577118,0.532188,0.162396,0.380959,0.179336,0.214468,0.196902,0.171923,0.402984,1.032137,0.406221,3521.635435,8510.655309


DataFrame salvo em ../results/metrics_1_1
+++++++++++ [1_2] | 1 ++++++++++++++++++
Stopped at epoch:  12
32/32 [==============================] - 0s 5ms/step
+++++++++++ [1_2] | 2 ++++++++++++++++++
Stopped at epoch:  6
32/32 [==============================] - 0s 3ms/step
+++++++++++ [1_2] | 3 ++++++++++++++++++
Stopped at epoch:  6
32/32 [==============================] - 0s 4ms/step
+++++++++++ [1_2] | 4 ++++++++++++++++++
Stopped at epoch:  6
32/32 [==============================] - 0s 4ms/step
+++++++++++ [1_2] | 5 ++++++++++++++++++
Stopped at epoch:  6
32/32 [==============================] - 0s 3ms/step
+++++++++++ [1_2] | 6 ++++++++++++++++++
Stopped at epoch:  6
32/32 [==============================] - 0s 4ms/step
+++++++++++ [1_2] | 7 ++++++++++++++++++
Stopped at epoch:  6
32/32 [==============================] - 0s 8ms/step
+++++++++++ [1_2] | 8 ++++++++++++++++++
Stopped at epoch:  6
32/32 [==============================] - 0s 8ms/step
+++++++++++ [1_2] | 9 +++++++++++++++

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic
model_1_2_24,0.828421,0.538104,0.775349,0.828143,0.818920,0.066328,0.274201,0.068869,0.065672,0.067270,0.108141,0.257543,1.013126,0.259611,3523.426278,8512.446153
model_1_2_23,0.821237,0.532211,0.768495,0.819560,0.811677,0.069105,0.277700,0.070970,0.068952,0.069961,0.110519,0.262879,1.013675,0.264991,3523.344243,8512.364117
model_1_2_22,0.817727,0.529670,0.765616,0.815209,0.808252,0.070462,0.279208,0.071853,0.070614,0.071234,0.111640,0.265448,1.013944,0.267580,3523.305352,8512.325227
model_1_2_21,0.814140,0.526764,0.762515,0.810886,0.804749,0.071849,0.280933,0.072804,0.072266,0.072535,0.112759,0.268047,1.014218,0.270200,3523.266373,8512.286248
model_1_2_20,0.810356,0.524023,0.759494,0.806030,0.801005,0.073312,0.282560,0.073730,0.074122,0.073926,0.113938,0.270762,1.014508,0.272936,3523.226066,8512.245940
model_1_2_19,0.806385,0.521459,0.757672,0.800493,0.797406,0.074847,0.284083,0.074288,0.076237,0.075263,0.115161,0.273582,1.014811,0.275779,3523.184620,8512.204494
model_1_2_18,0.802549,0.518572,0.754561,0.795700,0.793657,0.076330,0.285797,0.075242,0.078069,0.076655,0.116313,0.276278,1.015105,0.278497,3523.145388,8512.165263
model_1_2_17,0.798582,0.515661,0.751872,0.790477,0.789862,0.077863,0.287524,0.076066,0.080065,0.078065,0.117537,0.279040,1.015408,0.281282,3523.105597,8512.125471
model_1_2_16,0.794571,0.512576,0.748491,0.785489,0.785901,0.079414,0.289356,0.077103,0.081971,0.079537,0.118730,0.281805,1.015715,0.284068,3523.066167,8512.086041
model_1_2_15,0.790519,0.509563,0.744617,0.780923,0.781954,0.080980,0.291145,0.078290,0.083716,0.081003,0.119902,0.284570,1.016025,0.286856,3523.027100,8512.046974


DataFrame salvo em ../results/metrics_1_2
+++++++++++ [1_3] | 1 ++++++++++++++++++
Stopped at epoch:  10
32/32 [==============================] - 0s 4ms/step
+++++++++++ [1_3] | 2 ++++++++++++++++++


...

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic
model_7_4_24,0.932067,0.601766,0.935363,0.905497,0.925831,0.026261,0.236409,0.029510,0.020440,0.024975,0.079737,0.162053,1.005197,0.163355,3525.279318,8514.299192
model_7_4_23,0.930673,0.601037,0.933966,0.903580,0.924269,0.026800,0.236842,0.030148,0.020854,0.025501,0.080560,0.163707,1.005303,0.165022,3525.238710,8514.258584
model_7_4_22,0.929251,0.600284,0.932540,0.901625,0.922674,0.027350,0.237289,0.030799,0.021277,0.026038,0.081391,0.165378,1.005412,0.166706,3525.198086,8514.217960
model_7_4_21,0.927801,0.599508,0.931090,0.899627,0.921050,0.027910,0.237749,0.031461,0.021709,0.026585,0.082230,0.167064,1.005523,0.168406,3525.157515,8514.177389
model_7_4_20,0.926321,0.598715,0.929611,0.897588,0.919392,0.028482,0.238220,0.032136,0.022150,0.027143,0.083076,0.168767,1.005636,0.170123,3525.116943,8514.136817
model_7_4_19,0.924813,0.597904,0.928103,0.895512,0.917703,0.029066,0.238702,0.032824,0.022599,0.027712,0.083931,0.170486,1.005752,0.171856,3525.076398,8514.096273
model_7_4_18,0.923274,0.597069,0.926569,0.893393,0.915983,0.029660,0.239197,0.033525,0.023058,0.028291,0.084793,0.172222,1.005869,0.173605,3525.035889,8514.055763
model_7_4_17,0.921703,0.596211,0.925001,0.891230,0.914225,0.030268,0.239707,0.034241,0.023526,0.028883,0.085665,0.173976,1.005990,0.175373,3524.995352,8514.015226
model_7_4_16,0.920102,0.595324,0.923408,0.889025,0.912437,0.030886,0.240233,0.034968,0.024002,0.029485,0.086544,0.175745,1.006112,0.177157,3524.954874,8513.974749
model_7_4_15,0.918469,0.594414,0.921783,0.886775,0.910613,0.031518,0.240774,0.035710,0.024489,0.030100,0.087432,0.177533,1.006237,0.178959,3524.914390,8513.934264


DataFrame salvo em ../results/metrics_7_4
+++++++++++ [7_5] | 1 ++++++++++++++++++
Stopped at epoch:  6
32/32 [==============================] - 0s 4ms/step
+++++++++++ [7_5] | 2 ++++++++++++++++++
Stopped at epoch:  6
32/32 [==============================] - 0s 4ms/step
+++++++++++ [7_5] | 3 ++++++++++++++++++
Stopped at epoch:  6
32/32 [==============================] - 0s 4ms/step
+++++++++++ [7_5] | 4 ++++++++++++++++++
Stopped at epoch:  6
32/32 [==============================] - 0s 5ms/step
+++++++++++ [7_5] | 5 ++++++++++++++++++
Stopped at epoch:  6
32/32 [==============================] - 0s 6ms/step
+++++++++++ [7_5] | 6 ++++++++++++++++++
Stopped at epoch:  6
32/32 [==============================] - 0s 5ms/step
+++++++++++ [7_5] | 7 ++++++++++++++++++
Stopped at epoch:  6
32/32 [==============================] - 0s 6ms/step
+++++++++++ [7_5] | 8 ++++++++++++++++++
Stopped at epoch:  6
32/32 [==============================] - 0s 7ms/step
+++++++++++ [7_5] | 9 ++++++++++++++++

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic
model_7_5_24,0.958937,0.613912,0.967170,0.955838,0.964119,0.015874,0.229199,0.014545,0.015589,0.015067,0.062512,0.125991,1.003141,0.127003,3526.286174,8515.306048
model_7_5_23,0.958098,0.613540,0.966479,0.954917,0.963367,0.016198,0.229419,0.014851,0.015914,0.015383,0.063140,0.127273,1.003206,0.128295,3526.245681,8515.265555
model_7_5_22,0.957240,0.613153,0.965770,0.953976,0.962598,0.016530,0.229649,0.015165,0.016246,0.015706,0.063776,0.128569,1.003271,0.129602,3526.205148,8515.225022
model_7_5_21,0.956365,0.612767,0.965050,0.953018,0.961815,0.016868,0.229878,0.015484,0.016584,0.016034,0.064417,0.129878,1.003338,0.130922,3526.164629,8515.184503
model_7_5_20,0.955478,0.612428,0.964326,0.952049,0.961026,0.017211,0.230079,0.015805,0.016927,0.016366,0.065055,0.131191,1.003406,0.132245,3526.124390,8515.144265
model_7_5_19,0.954570,0.612059,0.963580,0.951052,0.960214,0.017562,0.230299,0.016135,0.017279,0.016707,0.065703,0.132522,1.003475,0.133586,3526.084041,8515.103915
model_7_5_18,0.953643,0.611678,0.962820,0.950029,0.959383,0.017920,0.230524,0.016472,0.017640,0.017056,0.066359,0.133867,1.003546,0.134942,3526.043641,8515.063515
model_7_5_17,0.952693,0.611218,0.962029,0.948986,0.958527,0.018288,0.230798,0.016823,0.018008,0.017415,0.067027,0.135232,1.003619,0.136318,3526.003067,8515.022942
model_7_5_16,0.951730,0.610883,0.961244,0.947920,0.957665,0.018660,0.230997,0.017170,0.018384,0.017777,0.067686,0.136602,1.003693,0.137699,3525.962733,8514.982607
model_7_5_15,0.950744,0.610430,0.960432,0.946830,0.956778,0.019041,0.231265,0.017530,0.018769,0.018149,0.068362,0.137990,1.003768,0.139098,3525.922303,8514.942178


DataFrame salvo em ../results/metrics_7_5
+++++++++++ [7_6] | 1 ++++++++++++++++++
Stopped at epoch:  8


KeyboardInterrupt: 